In [1]:
from tractools import run_tractography
from bossHandler import bossHandler
from tractography import vertices
from intern.resource.boss.resource import *
from intern.remote.boss import BossRemote
from skimage import filters
import numpy as np
from sklearn.mixture import GaussianMixture
from scipy.stats import norm
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.mlab as mlab
import glob
import skfmm
from scipy.ndimage.filters import laplace
from skimage.morphology import binary_opening, binary_closing, binary_dilation
from skimage.morphology import skeletonize_3d,label
from scipy.ndimage.morphology import *
from tifffile import imsave
from skimage import img_as_ubyte, img_as_uint, color

# Parameters:
coll_name = 'ailey-dev'
exp_name = 'DRN-BLA_2378'
chan_name = 'Ch0'

/usr/local/lib/python2.7/dist-packages/tifffile/tifffile.py:306: UserWarning: ImportError: No module named '_tifffile'. Loading of some compressed images will be very slow. Tifffile.c can be obtained at http://www.lfd.uci.edu/~gohlke/
  "ImportError: No module named '_tifffile'. "


In [2]:
bHandler = bossHandler(coll_name)
bHandler.select_experiment(exp_name)
bHandler.select_channel(chan_name)
coor = bHandler.get_coordinate_frame()

x_range = coor.x_stop
y_range = coor.y_stop
z_range = coor.z_stop


In [3]:
x_slices = range(x_range)
x_slices = np.array(x_slices[0::500] + [x_range + 1])
x_slices = zip(x_slices , np.roll(x_slices-1,-1))[0:-1]


y_slices = range(y_range)
y_slices = np.array(y_slices[0::500] + [y_range + 1])
y_slices = zip(y_slices , np.roll(y_slices-1,-1))[0:-1]


z_slices = range(z_range)
z_slices = np.array(z_slices[0::100] + [z_range + 1])
z_slices = zip(z_slices , np.roll(z_slices-1,-1))[0:-1]



In [4]:
def get_filename(xx, yy, zz):
    return 'x-'+str(xx[0])+'-'+str(xx[1])+'_y-'+str(yy[0])+'-'+str(yy[1])+'_z-'+str(zz[0])+'-'+str(zz[1])+'.tiff'

In [5]:
# Helper functions for ingestation (C: vikram)
def get_channel_resource(rmt, chan_name, coll_name, exp_name, type='image', base_resolution=0, sources=[], datatype='uint16', new_channel=True):
    channel_resource = ChannelResource(chan_name, coll_name, exp_name, type=type, 
                                    base_resolution=base_resolution, sources=sources, datatype=datatype)
    if new_channel: 

        new_rsc = rmt.create_project(channel_resource)
        return new_rsc

    return channel_resource



def upload_to_boss(rmt, data, channel_resource, resolution=0):
    Z_LOC = 0
    size = data.shape
    for i in range(0, data.shape[Z_LOC], 16):
        last_z = i+16
        if last_z > data.shape[Z_LOC]:
            last_z = data.shape[Z_LOC]
        print(resolution, [0, size[2]], [0, size[1]], [i, last_z])
        rmt.create_cutout(channel_resource, resolution, [0, size[2]], [0, size[1]], [i, last_z], np.asarray(data[i:last_z,:,:], order='C'))
        

In [6]:
rmt = BossRemote()
new_coll_name = 'tractography'
new_exp_name = 'DRN-BLA_2378'
new_chan_name = 'Ch0'


for xx in x_slices:
    for yy in y_slices:
        for zz in z_slices:
            data_cutout_raw = bHandler.get_cutout(list(xx) , list(yy), list(zz))
            data_cutout_raw = np.transpose(img_as_uint(data_cutout_raw),(1,2,0))
            skeleton = run_tractography(data_cutout_raw)
            filename = get_filename(xx,yy,zz)
            #save
            imsave('/run/mount/DRN-BLA_2378_2p_glycerol_Ch0/output_DRN_BLA_2378/' + filename , skeleton)
            #upload_to_boss
#             chan_resource = rmt.get_channel(chan_name= new_chan_name, coll_name=new_coll_name, exp_name=new_exp_name)
#             rmt.create_cutout(channel_resource, 0, [xx[0], xx[1]], [yy[0], yy[1]], [zz[0], zz[1]], skeleton)
            print('done: ' + filename)

            
            

done: x-0-499_y-0-499_z-0-99.tiff
done: x-0-499_y-0-499_z-100-199.tiff


KeyboardInterrupt: 

In [ ]:
# x_rng = [9250, 9750]
# y_rng = [18000, 18500]
# z_rng = [500, 600]
# test_lbl = '4'

# # Using bossHandler class to download from Boss:
# bHandler = bossHandler(coll_name)
# bHandler.select_experiment(exp_name)
# exp = bHandler.get_experiment()
# bHandler.list_channels()
# bHandler.select_channel(chan_name)
# coor = bHandler.get_coordinate_frame()
# data_cutout_raw = bHandler.get_cutout(x_rng, y_rng, z_rng)
# data_cutout_raw = np.transpose(img_as_uint(data_cutout_raw),(1,2,0))

In [ ]:
# skeleton = run_tractography(data_cutout_raw)